In [1]:
%matplotlib qt

import matplotlib.pyplot as plt
from poisson_fem import PoissonFEM
import ROM
import GenerativeSurrogate as gs
import Data as dta
import numpy as np
import scipy.sparse as sps
import scipy.sparse.linalg as lg
import time
import petsc4py
import sys
petsc4py.init(sys.argv)
from petsc4py import PETSc
import torch
from torch import optim

/home/constantin/anaconda3/envs/genDRROM/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/constantin/anaconda3/envs/genDRROM/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/constantin/anaconda3/envs/genDRROM/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/c

In [2]:
# Some fixed parameters
lin_dim_rom = 4                      # Linear number of rom elements
a = np.array([1, 2, 3])              # Boundary condition function coefficients
dim_z = 30                            # Latent space dimension
dtype = torch.float                  # Tensor data type

In [3]:
# Define mesh and boundary conditions
mesh = PoissonFEM.RectangularMesh(np.ones(lin_dim_rom)/lin_dim_rom)
# mesh.plot()

def origin(x):
    return np.abs(x[0]) < np.finfo(float).eps and np.abs(x[1]) < np.finfo(float).eps

def ess_boundary_fun(x):
    return 0.0
mesh.set_essential_boundary(origin, ess_boundary_fun)

def domain_boundary(x):
    # unit square
    return np.abs(x[0]) < np.finfo(float).eps or np.abs(x[1]) < np.finfo(float).eps or \
            np.abs(x[0]) > 1.0 - np.finfo(float).eps or np.abs(x[1]) > 1.0 - np.finfo(float).eps
mesh.set_natural_boundary(domain_boundary)

def flux(x):
    q = np.array([a[0] + a[2]*x[1], a[1] + a[2]*x[0]])
    return q

In [4]:
# Set up solver
ksp = PETSc.KSP().create()
ksp.setType('preonly')
precond = ksp.getPC()
precond.setType('cholesky')
ksp.setFromOptions() #???

In [5]:
#Spepify right hand side and stiffness matrix
#Define boundary flux field
rhs = PoissonFEM.RightHandSide(mesh)
rhs.set_natural_rhs(mesh, flux)
funSpace = PoissonFEM.FunctionSpace(mesh)
K = PoissonFEM.StiffnessMatrix(mesh, funSpace, ksp)
rhs.set_rhs_stencil(mesh, K)

In [6]:
# define rom
rom = ROM.ROM(mesh, K, rhs)

In [7]:
trainingData = dta.StokesData(range(256))
trainingData.read_data(['IMG'])
# trainingData.plotMicrostruct(1)
trainingData.reshape_microstructure_image()

In [8]:
model = gs.GenerativeSurrogate(rom, trainingData, dim_z)

In [9]:
steps = int(1)
for s in range(steps):
    print('step = ', s)
    batch_samples_pf = torch.multinomial(torch.ones(trainingData.n_samples_in), model.batch_size_N_pf)
#     batch_samples_pc = torch.multinomial(torch.ones(trainingData.n_samples_out), model.batch_size_N_pc)
    model.opt_latent_dist_step()
    model.pf_step(batch_samples_pf)
#     model.pc_step(batch_samples_pc)
    

step =  0
loss_pf =  tensor(11633114., grad_fn=<SubBackward0>)


In [10]:
model.plot_input_reconstruction()
f = plt.gcf()
f.suptitle('Untrained, N = 1184', fontsize=32, y=.7)

Text(0.5, 0.7, 'Untrained, N = 1184')

tensor([-2.8593e+04, -6.2677e+03, -5.6672e+04, -8.9069e+03, -1.7516e+04,
        -3.1449e+04, -7.1716e+04, -1.7030e+04, -7.8943e+03, -3.3994e+04,
        -5.1749e+04, -3.1313e+04, -5.7585e+03, -8.1368e+04, -1.9051e+05,
        -7.9617e+12], dtype=torch.float64)

tensor([ 4.9002e-02, -2.6601e-01,  1.1851e+00,  1.9921e+00,  5.2263e-01,
         1.0655e+00,  1.3936e+00,  3.2169e+00,  3.7125e+00,  1.2641e+00,
         2.0406e+00,  3.1584e+00,  4.9844e+00,  5.5034e+00,  1.8411e+00,
         2.7545e+00,  4.6515e+00,  5.9092e+00,  7.4918e+00,  2.4414e+00,
         3.9609e+00,  8.1447e+00,  1.2162e+01,  1.8518e+04],
       dtype=torch.float64, grad_fn=<AutogradROMBackward>)

tensor([-1176.5820,  -360.1528,  -387.1885,  -116.6333,   -81.0118,  -384.6716,
         -243.3890,  -138.1477,  -404.8069, -1034.5117,  -575.3208,  -184.8699,
        -1597.8018,  -466.5190,   -63.2471,  -310.9463], dtype=torch.float64)